In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mesa.space import MultiGrid
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

In [2]:
'''Recreating user submitted model found on NetLogo- "Artificial Financial Markets"'''

'Recreating user submitted model found on NetLogo- "Artificial Financial Markets"'

In [3]:
class FinancialAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.number_of_shares = 1
        self.opinion_vol= self.model.sigma + random.uniform(0, 0.1) #sigma is interactive parameter
        self.news_sensitivity= random.uniform(0, self.model.maximum_news_sensitivity) #maximum_news_sensitivity is also an interactive parameter
        self.base_propensity_to_sentiment_contagion= random.uniform(0, self.model.maximum_base_propensity_to_sentiment_contagion)
        # maximum_base_propensity_to_sentiment_contagion is also an interactive parameter
        self.propensity_to_sentiment_contagion = self.base_propensity_to_sentiment_contagion
        self.my_sentiment=0
        
    def step(self):
        sentiment= [agent.my_sentiment for agent in self.model.grid.get_neighbors(self.pos,moore=True,include_center=False,radius=1)]
        if((self.propensity_to_sentiment_contagion* sum (sentiment)+ self.news_sensitivity *news_arrival(FinancialMarket) + np.random.normal(self.model.miu,self.opinion_vol))> 0):
          #miu is also an interactive parameter
            self.my_sentiment= 1
            self.number_of_shares += 1 #Buy
        else:
            self.my_sentiment = -1
            self.number_of_shares -=  1 #Sell
            
        #updating market sentiment
        if (self.model.returns> 0):
            if (news_arrival(FinancialMarket) > 0):
                self.propensity_to_sentiment_contagion += self.model.returns
            elif (news_arrival(FinancialMarket) < 0):
                self.propensity_to_sentiment_contagion -= self.model.returns
        elif (self.model.returns < 0):
            if (news_arrival(FinancialMarket) < 0):
                self.propensity_to_sentiment_contagion -= self.model.returns
            elif (news_arrival(FinancialMarket) > 0):
                self.propensity_to_sentiment_contagion += self.model.returns
       
                       

In [4]:
def news_arrival(model):
    if(np.random.normal(0,1) > 0):
         news_qualitative_meaning =1 
    else: 
        news_qualitative_meaning= -1
    return news_qualitative_meaning
         
   

In [5]:

def market_clearing(model):
    number_of_traders= model.num_agents
    returns=sum([agent.my_sentiment for agent in model.schedule.agents]) / number_of_traders
    try:
        log_price =model.log_price+ returns
    except:
         log_price= 0 + returns
    return returns,log_price

In [6]:
'''def returns(model):
    number_of_traders= model.num_agents
    returns=sum([agent.my_sentiment for agent in model.schedule.agents]) / number_of_traders
    return returns'''
    
    

'def returns(model):\n    number_of_traders= model.num_agents\n    returns=sum([agent.my_sentiment for agent in model.schedule.agents]) / number_of_traders\n    return returns'

In [7]:
'''def logprice(model):
    try:
        log_price =model.log_price+ model.returns
    except:
         log_price= 0 + model.returns
    return log_price'''

'def logprice(model):\n    try:\n        log_price =model.log_price+ model.returns\n    except:\n         log_price= 0 + model.returns\n    return log_price'

In [8]:
#do this later
def compute_volatility_indicator(model):
    volatility_indicator= abs(returns)
    return volatility_indicator

In [9]:
class FinancialMarket(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height,sigma,maximum_news_sensitivity,maximum_base_propensity_to_sentiment_contagion,miu):
        self.running = True
        self.num_agents = N
        self.sigma=sigma
        self.maximum_news_sensitivity= maximum_news_sensitivity
        self.maximum_base_propensity_to_sentiment_contagion= maximum_base_propensity_to_sentiment_contagion
        self.miu=miu
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.returns=0
        # Create agents
        for i in range(self.num_agents):
            a = FinancialAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            
        self.datacollector = DataCollector(
            model_reporters={"News-Arrival": news_arrival,"Market-Clearing":market_clearing}
            #model_reporters={"News-Arrival": news_arrival,"Returns":returns,"logprice":logprice}
            #agent_reporters ={"Wealth":"my_sentiment"}
        )
        
        
    def step(self):
        self.datacollector.collect(self)
        a=self.datacollector.get_model_vars_dataframe()           
        b=a.iloc[-1,0]
        self.returns=b[0]
        self.log_price=b[1]
        self.datacollector.collect(self)
        #self.returns=a[-1,1]
        #self.log_price=a[-1,2]        
        self.schedule.step()
        #print(self.datacollector.model_vars['Market-Clearing'][-1][0])

In [10]:
#Run the model a certain number of times: in this case 20 times
model = FinancialMarket(50, 10, 10,0.4,0.7,0.6,0)
for i in range(20):
    model.step()
   

In [11]:
output = model.datacollector.get_model_vars_dataframe()
type(model.datacollector.model_vars["Market-Clearing"][-1])

tuple

In [12]:
output

,Market-Clearing,News-Arrival
0,"(0.0, 0.0)",1
1,"(0.0, 0.0)",1
2,"(0.08, 0.08)",-1
3,"(0.08, 0.16)",1
4,"(0.2, 0.28)",-1
5,"(0.2, 0.48000000000000004)",-1
6,"(0.32, 0.6000000000000001)",-1
7,"(0.32, 0.9200000000000002)",1
8,"(-0.12, 0.4800000000000001)",-1
9,"(-0.12, 0.3600000000000001)",1


In [15]:
# server.py
#Plot agents in their grid and show their movement
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import ChartModule
from mesa.visualization.UserParam import UserSettableParameter



n_slider1 = UserSettableParameter('slider', "Number of Agents", 100, 2, 200, 1)
n_slider2 = UserSettableParameter('slider', "sigma", 0.4, 0, 1, 0.001)
n_slider3 = UserSettableParameter('slider', "Maximum News Sensitivity", 0.7, 0, 1, .01)
n_slider4 = UserSettableParameter('slider', "Maximum Propensity to Contagion", 0.6, 0, 1, .01)
n_slider5 = UserSettableParameter('slider', "Bias", 0, -1, 1, .01)




chart = ChartModule([{"Label": "News-Arrival",
                      "Color": "Black"},{"Label": "Market-Clearing",
                      "Color": "Red"}],
                    data_collector_name='datacollector')

def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    if agent.my_sentiment > 0:
        portrayal["Color"] = "green"
    else:
        portrayal["Color"] = "red"
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(FinancialMarket,
                       [grid,chart],
                       "Simulated Financial Market",
                       {"N": n_slider1, "width": 10, "height": 10,"sigma":n_slider2,"maximum_news_sensitivity":n_slider3,"maximum_base_propensity_to_sentiment_contagion":n_slider4,"miu":n_slider5})

{"type":"get_step","step":251}


In [17]:
# run.py
#from server import server
server.port = 8533 # The default
server.launch()

Interface starting at http://127.0.0.1:8533
{"type":"get_step","step":259}
{"type":"get_step","step":260}
{"type":"get_step","step":261}
{"type":"get_step","step":262}
Socket opened!
{"type":"get_step","step":263}
{"type":"get_params"}
{"type":"reset"}
{"type":"get_step","step":264}
{"type":"get_step","step":265}
{"type":"get_step","step":1}
{"type":"get_step","step":266}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":267}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":268}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":269}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":270}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":271}
{"type":"get_step","step":16}
{"type":"get_step","step":17